# Connecting to PostgreSQL 

Example of connecting to PostgreSQL databases from Python using the [`psycopg2`](http://initd.org/psycopg/) package.

`psycopg2` is the most popular package for connecting to PostgreSQL databases.  There are frameworks, such as [`sqlalchemy`](https://www.sqlalchemy.org/), built on top of it that add additional functionality and ways of working with databases.  

You need to have installed `psycopg2` before running this notebook.

The packages for connecting to other types of SQL databases have a similar syntax and workflow.  They are governed by the [Python Database API Specification](https://www.python.org/dev/peps/pep-0249/).

In [ ]:
# install if needed
!pip install psycopg2-binary

In [1]:
import psycopg2

## Connect

Fill in the connection information below.

Note that in scripts that you write, especially those that you share, you probably don't want to hard code passwords into your file.  Alternatives are to read in an environment variable, read values from a configuration file that you don't share (just share a blank template), or include a step in your script or application to read the password from a command line argument or input from the user.

In [2]:
# example connection: update with your specifics
conn = psycopg2.connect(dbname="workshop", host="localhost", 
                        user="myusername", password="mypassword")

## Get a Cursor

You need a cursor to execute queries.

In [3]:
cur = conn.cursor()

## Execute Statements

Using the cursor, you can then execute SQL statements.  By default, statements are part of a transaction and are not permanent until you commit (or you can rollback) the connection. 

In [4]:
cur.execute("""CREATE TABLE test (id serial PRIMARY KEY, 
            num float, name text);""")

Insert some random data

In [5]:
import random
from string import ascii_lowercase
for val in ascii_lowercase:
    cur.execute("INSERT INTO test (num, name) VALUES (%s, %s)", 
                (random.random(), val))

Above, val will get quoted automatically when inserted into the statement.

Now, make sure to commit

In [6]:
cur.execute("select * from test")
cur.fetchall()

[(1, 0.474799792173992, 'a'),
 (2, 0.4343197397984, 'b'),
 (3, 0.734075711878846, 'c'),
 (4, 0.574939913433025, 'd'),
 (5, 0.648294282823083, 'e'),
 (6, 0.533967073414023, 'f'),
 (7, 0.14715557097833, 'g'),
 (8, 0.496855813127589, 'h'),
 (9, 0.212784192378674, 'i'),
 (10, 0.0284934256490874, 'j'),
 (11, 0.75534720558168, 'k'),
 (12, 0.912260279041763, 'l'),
 (13, 0.857371521730459, 'm'),
 (14, 0.547891220371794, 'n'),
 (15, 0.043662623849676, 'o'),
 (16, 0.0582995647720028, 'p'),
 (17, 0.738787678610912, 'q'),
 (18, 0.808302296259653, 'r'),
 (19, 0.200124286398337, 's'),
 (20, 0.984151212129189, 't'),
 (21, 0.733976443304754, 'u'),
 (22, 0.265577206110272, 'v'),
 (23, 0.863835908047373, 'w'),
 (24, 0.646911543436201, 'x'),
 (25, 0.101474656849782, 'y'),
 (26, 0.661636446923057, 'z')]

In [7]:
conn.commit() ## conn, not cur

## Getting Results

You fetch results back into Python.

In [8]:
cur.execute("SELECT * FROM test;")
print(cur.fetchone()) ## get one result (you get tuples)

(1, 0.474799792173992, 'a')


Once that first row is fetched, it's not the result set anymore.  If we pull more results, the first row won't be included.  This means also that if we re-run a fetch command, previously fetched results will be gone.

In [9]:
for row in cur.fetchall(): ## fetch everything still in the result set
    print("ID: {}\nNUM: {};\nNAME: {}\n".format(*row)) # * unpacks the row tuple

ID: 2
NUM: 0.4343197397984;
NAME: b

ID: 3
NUM: 0.734075711878846;
NAME: c

ID: 4
NUM: 0.574939913433025;
NAME: d

ID: 5
NUM: 0.648294282823083;
NAME: e

ID: 6
NUM: 0.533967073414023;
NAME: f

ID: 7
NUM: 0.14715557097833;
NAME: g

ID: 8
NUM: 0.496855813127589;
NAME: h

ID: 9
NUM: 0.212784192378674;
NAME: i

ID: 10
NUM: 0.0284934256490874;
NAME: j

ID: 11
NUM: 0.75534720558168;
NAME: k

ID: 12
NUM: 0.912260279041763;
NAME: l

ID: 13
NUM: 0.857371521730459;
NAME: m

ID: 14
NUM: 0.547891220371794;
NAME: n

ID: 15
NUM: 0.043662623849676;
NAME: o

ID: 16
NUM: 0.0582995647720028;
NAME: p

ID: 17
NUM: 0.738787678610912;
NAME: q

ID: 18
NUM: 0.808302296259653;
NAME: r

ID: 19
NUM: 0.200124286398337;
NAME: s

ID: 20
NUM: 0.984151212129189;
NAME: t

ID: 21
NUM: 0.733976443304754;
NAME: u

ID: 22
NUM: 0.265577206110272;
NAME: v

ID: 23
NUM: 0.863835908047373;
NAME: w

ID: 24
NUM: 0.646911543436201;
NAME: x

ID: 25
NUM: 0.101474656849782;
NAME: y

ID: 26
NUM: 0.661636446923057;
NAME: z



Note that results are unnamed tuples.  We have to keep track ourselves of what value is in what position in the results.  We can get a list of column names if we need:

In [10]:
colnames = [desc[0] for desc in cur.description]
print(colnames)

['id', 'num', 'name']


Or we could use `pandas` (see below).

## RISK: SQL Injection

If you're not careful, if you accept values from a user and use them directly in a database query, a malicious user can insert code to harm your database.  To avoid this, don't construct SQL queries as strings on your own.  Instead, use the built in functionality to put placeholders for values in the query, and then send a tuple with Python objects that the values will be read from.  The latter way, the values get sanitized.

For more details, see [`psycopg2` documentation](http://initd.org/psycopg/docs/usage.html#passing-parameters-to-sql-queries).

![xkcd sql injection](https://imgs.xkcd.com/comics/exploits_of_a_mom.png)
Source: https://xkcd.com/327/

In [ ]:
# BAD!!!!

myval = 3
cur.execute("select * from test where id="+str(myval)+";")
print(cur.fetchone())

In [11]:
# CORRECT

myval = 3
cur.execute("select * from test where id=%s;", [myval]) ## use a list or tuple for values
print(cur.fetchone())

(3, 0.734075711878846, 'c')


## Close Cursor and Connection

They will get closed when your script terminates, but it's good practice anyway.

In [12]:
cur.close()
conn.close()

## Notes

There are methods for copying data to the database from a file and copying a table from the database to a file.  See the `psycopg2` documentation for details and examples.

# Pandas

You can read data directly into a `pandas` data frame too.

In [13]:
import pandas as pd

In [14]:
# change the specific here for your use case
conn = psycopg2.connect(dbname="workshop", host="localhost", 
                        user="myusername", password="mypassword")

In [15]:
df = pd.read_sql_query("select * from test;", conn)
df

,id,num,name
0,1,0.474800,a
1,2,0.434320,b
2,3,0.734076,c
3,4,0.574940,d
4,5,0.648294,e
5,6,0.533967,f
6,7,0.147156,g
7,8,0.496856,h
8,9,0.212784,i
9,10,0.028493,j


There are functions for creating tables from pandas data frames, but these currently only work when using SQLite unless you are also using `sqlalchemy`.

In [16]:
conn.close()